In [5]:
import altair as alt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

In [6]:
files = sorted(list(map(lambda x: (int(x.name.split('_')[1].split('.')[0]), x), list(Path('.').glob('*.csv')))))

data = []
for dist, file in files:
    df = pd.read_csv(file, header=None)
    for _, (t, v, us) in df.iterrows():
        data.append((t, dist, 1/v, us))

In [7]:
df = pd.DataFrame(data, columns=["t", "dist", "IR", 'Ultrasound'])

sensor = 'Ultrasound'
(alt.Chart(df).encode(
    x=alt.X('dist:Q', title='Distance, cm'),
    y=alt.Y(f'min({sensor}):Q', scale=alt.Scale(zero=False), title='Boxplot, mV' if sensor == 'IR' else 'Boxplot, cm'),
    y2=f'max({sensor}):Q',
    tooltip=['dist', f'min({sensor})', f'max({sensor})']
).mark_rule() + alt.Chart(df).encode(
    x='dist:Q',
    y=f'mean({sensor}):Q',
    tooltip=['dist', f'min({sensor})', f'max({sensor})']
).mark_point(size=1)).properties(
    title=f'Correspondence between distance and {sensor} measurements'
)

/opt/homebrew/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [8]:
df = pd.DataFrame(data, columns=["t", "dist", "IR", 'Ultrasound'])
df['diff'] = df['dist'] - df['Ultrasound']

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(df['dist'].values.reshape(-1, 1), df['diff'].values)
diff_pred = model.predict(df['dist'].values.reshape(-1, 1))
df['diff_calibrated'] =  df['Ultrasound'] + diff_pred - df['dist']

In [9]:
def get_error_chart(sensor):
    return (alt.Chart(df).encode(
        x=alt.X('dist:Q', title='Distance, cm'),
        y=alt.Y(f'min({sensor}):Q', scale=alt.Scale(zero=False), title='Boxplot, mV' if sensor == 'IR' else 'Boxplot, cm'),
        y2=f'max({sensor}):Q',
        tooltip=['dist', f'min({sensor})', f'max({sensor})']
    ).mark_rule() + alt.Chart(df).encode(
        x='dist:Q',
        y=f'median({sensor}):Q',
        tooltip=['dist', f'min({sensor})', f'max({sensor})']
    ).mark_point(size=1)).properties(
        title=f'Error rate between Ultrasound and reference'
    )
    
get_error_chart('diff') | get_error_chart('diff_calibrated')

/opt/homebrew/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [10]:
mean

NameError: name 'mean' is not defined

In [ ]:
# dist, mv, _ = list(zip(*data))
# np.poly
dist = df.groupby('dist').mean().reset_index()['dist'].values[:35//2]
mv = df.groupby('dist').mean().reset_index()['IR'].values[:35//2]

In [ ]:
coeff = np.polyfit(dist, mv, deg=2)

plt.plot(dist, mv)
plt.plot(dist, np.polyval(coeff, dist))

In [ ]:
a,b = np.polyfit(np.log(dist), np.log(mv), deg=1)

plt.plot(dist, mv)
plt.plot(dist, np.exp(b + a*np.log(dist)))
print(a, b)